In [2]:
# Load 4 cluster network
import pypsa
import pandas as pd
# from Model_Code.heat_sector import add_oil_boiler_and_res_heat_demand_simple


# heat_demand_df = pd.read_csv("data/res_heat_demand.csv", index_col="snapshot", parse_dates=True)


# Heating sector added to same network
# n = add_oil_boiler_and_res_heat_demand_simple(n, heat_demand_df)



In [85]:
# n = pypsa.Network("D:/project_h2/Results/4/2020.nc")
n = pypsa.Network("D:/project_h2/Results/4/2021.nc")
# n2 = pypsa.Network("D:/project_h2/Results/4/2022.nc")

INFO:pypsa.io:Imported network 2021.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [4]:
biomass_addition_df=pd.read_csv("D:/project_h2/Networks/elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL/biomass_basic_addition_heat.csv", index_col=0)

In [5]:
for bus in ['DE0 2','DE0 9','DE0 A','DE0 E',]:
    bus_key = f"{bus} biomass"
    if bus_key in biomass_addition_df.index:
        p_nom = biomass_addition_df.loc[bus_key, "p_nom"]
        print(p_nom)

6386.18
3999.3
7686.63
3805.48


In [68]:
# Find all gas generators
gas_generators = n.generators[n.generators.carrier == 'gas']

# Get their power output
gas_gen_energy = n.generators_t.p[gas_generators.index].sum().sum()  # in MWh

# CO2 emissions
co2_factor_gas = n.carriers.loc['gas', 'co2_emissions']
gas_gen_emissions = gas_gen_energy * co2_factor_gas
print(f"Gas Generator CO₂ emissions: {gas_gen_emissions:.2f} tCO₂")


Gas Generator CO₂ emissions: 55120097.63 tCO₂


In [33]:
biomass_p0 = n.links_t.p0[biomass_links.index]


In [69]:
# Find all gas generators
gas_generators = n.generators[n.generators.carrier == 'gas']

# Get their power output
gas_gen_energy = n.generators_t.p[gas_generators.index].sum().sum()  # in MWh

# CO2 emissions
co2_factor_gas = n.carriers.loc['gas', 'co2_emissions']
gas_gen_emissions = gas_gen_energy * co2_factor_gas
print(f"Gas Generator CO₂ emissions: {gas_gen_emissions:.2f} tCO₂")


Gas Generator CO₂ emissions: 55120097.63 tCO₂


In [70]:
# Find all gas burners
gas_burners = n.links[n.links.carrier == 'gas_burner']

# Get their input energy from bus0 (in PyPSA, it's negative for input)
gas_burner_inputs = n.links_t.p0[gas_burners.index]  # p0 = input side

# Sum of absolute values (since p0 is negative)
gas_burner_energy = -gas_burner_inputs.sum().sum()  # in MWh

# Same CO₂ emission factor
gas_burner_emissions = gas_burner_energy * co2_factor_gas
print(f"Gas Burner CO₂ emissions: {gas_burner_emissions:.2f} tCO₂")


Gas Burner CO₂ emissions: -3827213.41 tCO₂


In [46]:
# Select links with names like "gas_burner" or using gas as input
gas_burner_links = [l for l in n.links.index if "gas_burner" in l]

# Get power drawn from bus0 (which is the gas bus)
gas_burner_input = n.links_t.p0[gas_burner_links]

# Total annual fuel used in MWh
total_gas_to_burners = -gas_burner_input.sum().sum()  # negative because p0 is input
print("Gas used by burners:", total_gas_to_burners, "MWh_th")


Gas used by burners: -20466402.709591445 MWh_th


In [ ]:
# Select all gas_burner links that actually exist in time series
oil_burner_links = [l for l in n.links.index if "oil_burner" in l and l in n.links_t.p0.columns]

if oil_burner_links:
    oil_burner_input = n.links_t.p0[oil_burner_links]
    total_oil_to_burners = -oil_burner_input.sum().sum()
    print("Gas used by burners:", total_oil_to_burners, "MWh_th")
else:
    print("No gas_burner links found in time series.")


Gas used by burners: -20466381.855350748 MWh_th


In [80]:
n.links[28:]

,bus0,bus1,carrier,geometry,length,p_nom,tags,under_construction,underground,p_min_pu,...,marginal_cost,bus2,efficiency2,type,lifetime,p_set,p_max_pu,terrain_factor,ramp_limit_up,ramp_limit_down
Link,,,,,,,,,,,,,,,,,,,,,
DE0 9 biomass_burner,DE0 9 biomass,DE0 9 heat,biomass_burner,,0.0,3999.30,,NaN,NaN,0.0,...,26.528,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 9 fixed_biomass_burner,DE0 9 biomass,DE0 9 heat,fixed_biomass_burner,,0.0,3999.30,,NaN,NaN,0.0,...,0.000,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 9 heat_pump,DE0 9,DE0 9 heat,heat_pump,,0.0,2000.00,,NaN,NaN,0.0,...,0.000,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 A biomass_burner,DE0 A biomass,DE0 A heat,biomass_burner,,0.0,7686.63,,NaN,NaN,0.0,...,26.528,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 A fixed_biomass_burner,DE0 A biomass,DE0 A heat,fixed_biomass_burner,,0.0,7686.63,,NaN,NaN,0.0,...,0.000,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 A heat_pump,DE0 A,DE0 A heat,heat_pump,,0.0,2000.00,,NaN,NaN,0.0,...,0.000,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 2 biomass_burner,DE0 2 biomass,DE0 2 heat,biomass_burner,,0.0,6386.18,,NaN,NaN,0.0,...,26.528,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 2 fixed_biomass_burner,DE0 2 biomass,DE0 2 heat,fixed_biomass_burner,,0.0,6386.18,,NaN,NaN,0.0,...,0.000,,1.0,,inf,0.0,1.0,1.0,NaN,NaN
DE0 2 heat_pump,DE0 2,DE0 2 heat,heat_pump,,0.0,2000.00,,NaN,NaN,0.0,...,0.000,,1.0,,inf,0.0,1.0,1.0,NaN,NaN


In [82]:
n.links[n.links.carrier == "oil_burner"]
n.carriers.loc["oil_burner"]


co2_emissions          0.248
color                       
nice_name                   
max_growth               inf
max_relative_growth      0.0
Name: oil_burner, dtype: object

In [83]:
n.carriers

,co2_emissions,color,nice_name,max_growth,max_relative_growth
Carrier,,,,,
biomass,0.000,#0c6013,Biomass,inf,0.0
OCGT,0.187,#d35050,Open-Cycle Gas,inf,0.0
offwind-dc,0.000,#74c6f2,Offshore Wind (DC),inf,0.0
offwind-ac,0.000,#6895dd,Offshore Wind (AC),inf,0.0
oil,0.248,#262626,Oil,inf,0.0
onwind,0.000,#235ebc,Onshore Wind,inf,0.0
solar,0.000,#f9d002,Solar,inf,0.0
coal,0.354,#707070,Coal,inf,0.0
CCGT,0.187,#b20101,Combined-Cycle Gas,inf,0.0


In [56]:
n2.links[n2.links.carrier == "oil_burner"][["bus0", "bus1", "p_nom", "p_nom_min", "p_nom_opt", "efficiency"]]


,bus0,bus1,p_nom,p_nom_min,p_nom_opt,efficiency
Link,,,,,,
DE0 9 oil_burner,DE0 9 oil,DE0 9 heat,6754.32,0.0,0.0,0.85
DE0 A oil_burner,DE0 A oil,DE0 A heat,12919.56,0.0,0.0,0.85
DE0 2 oil_burner,DE0 2 oil,DE0 2 heat,10722.84,0.0,0.0,0.85
DE0 E oil_burner,DE0 E oil,DE0 E heat,6396.07,0.0,0.0,0.85


In [11]:
removal_biomass_df = pd.read_csv("D:/project_h2/Networks/elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL/biomass_basic_removal_heat.csv")


In [61]:
# Filter only rows with decommissioning in the current year
expired = removal_biomass_df[removal_biomass_df["year_removed"] == 2021]

for _, row in expired.iterrows():
    link_name = f"{row['bus']} oil_burner"
    # print(link_name)
    if link_name in n.links.index:
        old_p_nom = n.links.at[link_name, "p_nom"]
        new_p_nom = max(0, old_p_nom - row["p_nom"])
        n.links.at[link_name, "p_nom"] = new_p_nom

        print(f"[{2021}] Removed {row['p_nom']} MW from {link_name} (new p_nom = {new_p_nom}) (old p_nom = {old_p_nom})")


[2021] Removed 277.66 MW from DE0 2 oil_burner (new p_nom = 11856.08) (old p_nom = 12133.74)
[2021] Removed 266.62 MW from DE0 9 oil_burner (new p_nom = 7331.990000000001) (old p_nom = 7598.610000000001)
[2021] Removed 569.38 MW from DE0 A oil_burner (new p_nom = 14035.34) (old p_nom = 14604.72)
[2021] Removed 271.82 MW from DE0 E oil_burner (new p_nom = 6958.5199999999995) (old p_nom = 7230.339999999999)


In [58]:
biomass_generators = n.generators[n.generators.carrier == "biomass"]
biomass_generator_names = biomass_generators.index

# Check total electricity produced by biomass generators
biomass_generation = n.generators_t.p[biomass_generator_names].sum().sum()
print("Electricity generation from biomass generators:", biomass_generation)


Electricity generation from biomass generators: 66028548.159264654


In [63]:
gas_links = n.links[n.links.carrier == "gas_burner"]
gas_link_names = gas_links.index

# Total thermal output from biomass burners (bus1)
gas_heat_output = n.links_t.p1[gas_link_names].sum().sum()
print("Heat delivered by gas burners:", gas_heat_output)


Heat delivered by gas burners: -18419743.669815674


In [47]:
# Fuel consumed by generators (electricity)
fuel_used_by_gen = n.generators_t.p[biomass_generator_names].sum()

# Fuel used by links (heat)
fuel_used_by_links = n.links_t.p0[biomass_link_names].sum()


In [49]:
fuel_used_by_links

Series([], dtype: float64)

In [64]:
heating_links = [
    name for name, row in n.links.iterrows()
    if 'heat' in str(row.get('bus1', '')) or 'heat' in str(row.get('bus2', ''))
]
heating_links

['DE0 2 CCGT',
 'DE0 9 CCGT',
 'DE0 A CCGT',
 'DE0 E CCGT',
 'DE0 9 biomass_burner',
 'DE0 9 fixed_biomass_burner',
 'DE0 9 heat_pump',
 'DE0 A biomass_burner',
 'DE0 A fixed_biomass_burner',
 'DE0 A heat_pump',
 'DE0 2 biomass_burner',
 'DE0 2 fixed_biomass_burner',
 'DE0 2 heat_pump',
 'DE0 E biomass_burner',
 'DE0 E fixed_biomass_burner',
 'DE0 E heat_pump',
 'DE0 9 gas_burner',
 'DE0 A gas_burner',
 'DE0 2 gas_burner',
 'DE0 E gas_burner',
 'DE0 9 oil_burner',
 'DE0 A oil_burner',
 'DE0 2 oil_burner',
 'DE0 E oil_burner']

In [308]:
heat_demand_df = pd.read_csv("data/res_heat_demand.csv", index_col='snapshot')

heat_demand_GWh = heat_demand_df.sum().sum() / 1000  # TWh
total_heat_cap = (heat_demand_GWh * 1e6) / (0.3 * 8760)  # MW

In [310]:
heat_demand_GWh

758999.9999973824

In [311]:
elec_demand_df = pd.read_csv("data/germany_electricity_load.csv", index_col='snapshot')

elec_demand_GWh = elec_demand_df.sum().sum() / 1000  # TWh
total_heat_cap = (elec_demand_GWh * 1e6) / (0.3 * 8760)  # MW

In [314]:
total_heat_cap

193743885.5571469

In [313]:
elec_demand_GWh

509158.93124418205

In [296]:
n1 = pypsa.Network("D:/project_h2/Results/4/2021.nc")


INFO:pypsa.io:Imported network 2021.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [65]:
n1.links['p_nom'][28:]


Link
DE0 9 biomass_burner              0.00
DE0 9 fixed_biomass_burner     3732.68
DE0 9 heat_pump                2000.00
DE0 A biomass_burner              0.00
DE0 A fixed_biomass_burner     7117.25
DE0 A heat_pump                2000.00
DE0 2 biomass_burner              0.00
DE0 2 fixed_biomass_burner     6108.52
DE0 2 heat_pump                2000.00
DE0 E biomass_burner              0.00
DE0 E fixed_biomass_burner     3533.66
DE0 E heat_pump                2000.00
DE0 9 gas_burner              20586.00
DE0 A gas_burner              40005.93
DE0 2 gas_burner              33141.42
DE0 E gas_burner              19798.80
DE0 9 oil_burner               7035.75
DE0 A oil_burner              13762.14
DE0 2 oil_burner              11287.20
DE0 E oil_burner               6674.16
Name: p_nom, dtype: float64

In [298]:
n2 = pypsa.Network("D:/project_h2/Results/4/2022.nc")

INFO:pypsa.io:Imported network 2022.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [327]:
saved_potential_heat = n.links.p_nom_max[n.links.p_nom_extendable == True].copy()
saved_potential_heat[22:]

Link
DE0 E electrolysis          inf
DE0 E fuel cell             inf
DE0 9 oil_burner            inf
DE0 A oil_burner            inf
DE0 2 oil_burner            inf
DE0 E oil_burner            inf
DE0 9 biomass_burner        inf
DE0 A biomass_burner        inf
DE0 2 biomass_burner        inf
DE0 E biomass_burner        inf
DE0 9 gas_burner            inf
DE0 A gas_burner            inf
DE0 2 gas_burner            inf
DE0 E gas_burner            inf
DE0 9 heat_pump         4910.22
DE0 A heat_pump         4910.22
DE0 2 heat_pump         4910.22
DE0 E heat_pump         4910.22
DE0 9 solar_thermal     4910.22
DE0 A solar_thermal     4910.22
DE0 2 solar_thermal     4910.22
DE0 E solar_thermal     4910.22
Name: p_nom_max, dtype: float64

In [275]:

heating_links = [
    name for name, row in n.links.iterrows()
    if 'heat' in str(row.get('bus1', '')) or 'heat' in str(row.get('bus2', ''))
]

for i in n.links.index[n.links.p_nom_extendable == True]:
    if i not in heating_links:
        continue

    tech_name = i.split()[-1]
    print(tech_name)

CCGT
CCGT
CCGT
CCGT
oil_burner
oil_burner
oil_burner
oil_burner
biomass_burner
biomass_burner
biomass_burner
biomass_burner
gas_burner
gas_burner
gas_burner
gas_burner
coal_burner
coal_burner
coal_burner
coal_burner
heat_pump
heat_pump
heat_pump
heat_pump
solar_thermal
solar_thermal
solar_thermal
solar_thermal


In [273]:
tech_name

'solar_thermal'

In [251]:
heating_links = [
        name for name, row in n.links.iterrows()
        if ('heat' in str(row.get('bus1', ''))) or ('heat' in str(row.get('bus2', '')))
    ]

In [11]:

n2 = pypsa.Network("D:/project_h2/Networks/elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL.nc")


INFO:pypsa.io:Imported network elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


following is the check on update_cost funtion where marginal cost of fossil generators is increased yearly, it should pick up all the gens whose carrier is present in fuel_cost.csv but it's only picking up H2 and not others

In [ ]:
fuel_cost = pd.read_csv('data/fuel_cost.csv')
fuels = list(fuel_cost.columns)
for tech in fuels:
    
    mask_link = n.links.carrier == tech
    mask_gen= (n.generators.carrier == tech) | (n.generators.carrier == f"imports_{tech}")
mask_gen[28:]

here it should show True for all fossil fuel generators

this is to check for co2update function, all the generators and links with following carriers should show up and True in the upcoming function but showing only for gas, not others
hence same goes for links as well, only gas boiler is showing as true which means only gas will be affected by co2update fucntion, so do gas generators

In [253]:

for carrier in list(n.carriers.index[n.carriers.co2_emissions > 0]):
    print(carrier)

OCGT
oil
coal
CCGT
lignite
gas


In [67]:

for carrier in list(n.carriers.index[n.carriers.co2_emissions > 0]):
    mask = n.links.carrier == carrier
mask[25:]

Link
DE0 E H2_input                False
DE0 E electrolysis            False
DE0 E fuel cell               False
DE0 9 biomass_burner          False
DE0 9 fixed_biomass_burner    False
DE0 9 heat_pump               False
DE0 A biomass_burner          False
DE0 A fixed_biomass_burner    False
DE0 A heat_pump               False
DE0 2 biomass_burner          False
DE0 2 fixed_biomass_burner    False
DE0 2 heat_pump               False
DE0 E biomass_burner          False
DE0 E fixed_biomass_burner    False
DE0 E heat_pump               False
DE0 9 gas_burner              False
DE0 A gas_burner              False
DE0 2 gas_burner              False
DE0 E gas_burner              False
DE0 9 oil_burner              False
DE0 A oil_burner              False
DE0 2 oil_burner              False
DE0 E oil_burner              False
Name: carrier, dtype: bool

In [46]:
n.generators

,p_nom_max,weight,p_nom,capital_cost,efficiency,marginal_cost,carrier,p_nom_extendable,bus,control,...,q_set,sign,lifetime,committable,start_up_cost,shut_down_cost,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,
DE0 2 biomass,2.185850e+00,0.000000,0.000000,273978.048251,0.468000,56.606084,biomass,True,DE0 2,Slack,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
DE0 2 coal,inf,0.000000,5982.573889,118311.880536,0.454788,80.490646,coal,False,DE0 2,,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
DE0 2 oil,inf,0.000000,198.700000,38234.561404,0.357667,187.324545,oil,False,DE0 2,,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
DE0 2 onwind,1.582678e+05,19690.248056,0.000000,143786.999475,1.000000,1.510698,onwind,True,DE0 2,,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
DE0 2 ror,0.000000e+00,2159.618510,0.000000,225783.929402,0.900000,0.009197,ror,True,DE0 2,,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE0 E biomass source,inf,NaN,0.000000,0.000000,1.000000,26.609339,biomass,True,DE0 E biomass,Slack,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
DE0 9 gas source,inf,NaN,0.000000,0.000000,1.000000,32.340000,gas,True,DE0 9 gas,PQ,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
DE0 A gas source,inf,NaN,0.000000,0.000000,1.000000,32.340000,gas,True,DE0 A gas,PQ,...,0.0,1.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0


In [38]:
for carrier in list(n.carriers.index[n.carriers.co2_emissions > 0]):
    mask = n.links.carrier.str.contains(carrier, case=False)
mask[28:]


Link
DE0 9 oil_burner        False
DE0 A oil_burner        False
DE0 2 oil_burner        False
DE0 E oil_burner        False
DE0 9 biomass_burner    False
DE0 A biomass_burner    False
DE0 2 biomass_burner    False
DE0 E biomass_burner    False
DE0 9 gas_burner         True
DE0 A gas_burner         True
DE0 2 gas_burner         True
DE0 E gas_burner         True
DE0 9 heat_pump         False
DE0 A heat_pump         False
DE0 2 heat_pump         False
DE0 E heat_pump         False
DE0 9 solar_thermal     False
DE0 A solar_thermal     False
DE0 2 solar_thermal     False
DE0 E solar_thermal     False
Name: carrier, dtype: bool

In [214]:
heating_links = []

for link_name, row in n.links.iterrows():
    if 'bus1' in row.index and isinstance(row['bus1'], str) and 'heat' in row['bus1']:
        heating_links.append(link_name)
    if 'bus2' in row.index and isinstance(row['bus2'], str) and 'heat' in row['bus2']:
        heating_links.append(link_name)

In [257]:
links=[]
for i in n.generators.index[n.generators.p_nom_extendable == True]:
    links.append(i)

In [260]:
electrical_tech_data = {
    "technology": ["heat_pump", "solar_thermal"],
    "capex": [800, 1100],
    "lifetime": [20, 25],
    "efficiency": [3, 1],
    "fom": [0.02, 0.01],
    "vom": [0, 0],
    "carrier": ["AC", "heat"],
    "cop": [3, None],
    "renewable": [1, 1],
    "p_nom_max": [60058, 55000],
    "p_nom_min": [0, 0]
}
electrical_df = pd.DataFrame(electrical_tech_data)

In [264]:
electrical_df[1:]

,technology,capex,lifetime,efficiency,fom,vom,carrier,cop,renewable,p_nom_max,p_nom_min
1,solar_thermal,1100,25,1,0.01,0,heat,NaN,1,55000,0


In [36]:
missing = [g for g in ['DE0 9 lignite', 'DE0 9 oil'] if g not in n.generators_t.p.columns]
print("Missing from generators_t.p:", missing)


Missing from generators_t.p: []


In [37]:
print(n.generators.loc[missing, ['p_nom', 'p_nom_max', 'p_nom_extendable', 'carrier']])


Empty DataFrame
Columns: [p_nom, p_nom_max, p_nom_extendable, carrier]
Index: []


In [62]:
import pypsa
import pandas as pd

# Load networks
n_2020 = pypsa.Network("Results/4/2020.nc")
n_2021 = pypsa.Network("Results/4/2021.nc")
n_scenario = pypsa.Network("C:/Users/86435/project_h2/Networks/elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL.nc")

# Extract marginal costs
df_marginals = pd.DataFrame({
    "2020": n_2020.generators["marginal_cost"],
    "2021": n_2021.generators["marginal_cost"],
    "Scenario": n_scenario.generators["marginal_cost"]
})

# Drop rows where all values are NaN
df_marginals.dropna(how='all', inplace=True)

# Save or print
df_marginals
df_marginals.to_csv("marginal_cost_comparison.csv")


INFO:pypsa.io:Imported network 2020.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network 2021.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


In [53]:
n1 = pypsa.Network("C:/Users/86435/project_h2/Networks/elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL.nc")
# Filter for oil and coal links (after conversion)
fossil_links = n1.links[n1.links.carrier.isin(["oil", "coal"])]

# Display core parameters
print("🔗 Fossil Links:\n")
print(fossil_links[["carrier", "marginal_cost", "p_nom_max", "build_year"]])



INFO:pypsa.io:Imported network elec_s_4_ec_lcopt_Co2L-1H-Ep-CCL.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


🔗 Fossil Links:

Empty DataFrame
Columns: [carrier, marginal_cost, p_nom_max, build_year]
Index: []


In [49]:
# Check time-dependent availability if p_max_pu exists
if hasattr(n1, "links_t") and hasattr(n1.links_t, "p_max_pu"):
    p_max_pu_fossil = n1.links_t.p_max_pu[fossil_links.index]
    print("\n⏱️  p_max_pu (time series availability):")
    print(p_max_pu_fossil.head())
else:
    print("⚠️  No time-dependent p_max_pu available for links.")


⏱️  p_max_pu (time series availability):
Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00]


In [224]:
# Optional: check old generators (if any remain)
fossil_gens = n.generators[n.generators.carrier.isin(["oil", "coal"])]
if not fossil_gens.empty:
    print("\n⚠️  Some fossil generators still exist (not converted):")
    print(fossil_gens[["carrier", "marginal_cost", "p_nom_max", "build_year"]])


⚠️  Some fossil generators still exist (not converted):
                  carrier  marginal_cost  p_nom_max  build_year
Generator                                                      
DE0 2 coal           coal     116.616607        inf           0
DE0 2 oil             oil     210.727758        inf           0
DE0 9 coal           coal     115.998760        inf           0
DE0 A coal           coal     117.110797        inf           0
DE0 A oil             oil     201.846825        inf           0
DE0 E coal           coal     116.105818        inf           0
DE0 E oil             oil     203.349332        inf           0
DE0 9 oil             oil     210.727758        inf           0
DE0 9 oil source      oil      97.172270        inf           0
DE0 A oil source      oil      97.172270        inf           0
DE0 2 oil source      oil      97.172270        inf           0
DE0 E oil source      oil      97.172270        inf           0
DE0 9 coal source    coal      81.418506       

In [ ]:
# Part of base_functions.py or a new helper file

def convert_fossil_generators_to_links(n, fuel_cost):
    """
    Converts oil, biomass, and coal generators into Links
    with associated fuel buses (no stores in this version).
    Applies only to AC buses.

    Parameters:
    -----------
    n : pypsa.Network
        The PyPSA network object to modify
    fuel_cost : pd.DataFrame
        DataFrame containing marginal fuel cost per carrier

    Returns:
    --------
    list
        Names of converted generators (removed from n.generators)
    """
    fossil_carriers = ["oil", "coal"]  # biomass skipped for now
    converted = []

    for bus in n.buses.index[n.buses.carrier == 'AC']:
        for carrier in fossil_carriers:
            gens = n.generators[(n.generators.bus == bus) & (n.generators.carrier == carrier)]
            if gens.empty:
                continue

            # Create fuel bus if not exists
            fuel_bus = f"{bus} {carrier}"
            if fuel_bus not in n.buses.index:
                n.add("Bus", fuel_bus, carrier=carrier,
                      x=n.buses.at[bus, 'x'], y=n.buses.at[bus, 'y'])

            # For each generator, replace with a Link
            for gen_name, row in gens.iterrows():
                link_name = f"{bus} {carrier}"
                n.add("Link", name=link_name,
                      bus0=fuel_bus, bus1=bus,
                      p_nom=row.p_nom,
                      p_nom_extendable=row.p_nom_extendable,
                      efficiency=row.efficiency,
                      capital_cost=row.get("capital_cost", 0.0),
                      marginal_cost=fuel_cost.loc[2020, carrier],
                      carrier=carrier)
                
                converted.append(gen_name)
                n.remove("Generator", gen_name)

    return converted


In [40]:
fuel_cost = pd.DataFrame({
    "coal": {2020: 22.0},
    "oil": {2020: 125.0}
})

In [41]:
convert_fossil_generators_to_links(n1, fuel_cost)


['DE0 2 oil',
 'DE0 2 coal',
 'DE0 9 coal',
 'DE0 A oil',
 'DE0 A coal',
 'DE0 E oil',
 'DE0 E coal']

In [66]:
fuel_cost = pd.read_csv("data/fuel_cost.csv")
fuels = fuel_cost.columns
for tech in fuels:
    mask_link = n.links.carrier == tech

In [70]:
n.carriers

,co2_emissions,color,nice_name,max_growth,max_relative_growth
Carrier,,,,,
biomass,0.000,#0c6013,Biomass,inf,0.0
OCGT,0.187,#d35050,Open-Cycle Gas,inf,0.0
offwind-dc,0.000,#74c6f2,Offshore Wind (DC),inf,0.0
offwind-ac,0.000,#6895dd,Offshore Wind (AC),inf,0.0
oil,0.248,#262626,Oil,inf,0.0
onwind,0.000,#235ebc,Onshore Wind,inf,0.0
solar,0.000,#f9d002,Solar,inf,0.0
coal,0.354,#707070,Coal,inf,0.0
CCGT,0.187,#b20101,Combined-Cycle Gas,inf,0.0


In [44]:
n1.links

,bus0,bus1,carrier,geometry,length,p_nom,tags,under_construction,underground,p_min_pu,...,efficiency,build_year,lifetime,p_nom_max,p_set,p_max_pu,marginal_cost,terrain_factor,ramp_limit_up,ramp_limit_down
Link,,,,,,,,,,,,,,,,,,,,,
T10,DE0 E,DE0 2,DC,"LINESTRING (11.629014 52.252137, 12.091596 48....",498.518178,2000.000000,"""name""=>""HVDC SuedOstLink"", ""ref""=>""https://ty...",True,False,-1.0,...,1.000000,0,inf,inf,0.0,1.0,0.0,1.0,NaN,NaN
T11,DE0 9,DE0 A,DC,"LINESTRING (7.206009 53.359403, 6.619451 51.27...",350.213475,2000.000000,"""name""=>""HVDC Line A-North"", ""ref""=>""https://t...",True,False,-1.0,...,1.000000,0,inf,inf,0.0,1.0,0.0,1.0,NaN,NaN
T18,DE0 9,DE0 2,DC,"LINESTRING (9.373979 53.922479, 9.117193 49.14...",591.654038,4000.000000,"""name""=>""HVDC SuedLink"", ""ref""=>""https://tyndp...",True,False,-1.0,...,1.000000,0,inf,inf,0.0,1.0,0.0,1.0,NaN,NaN
T20,DE0 A,DE0 2,DC,"LINESTRING (6.619451 51.272935, 8.458036 49.23...",347.522410,600.000000,"""name""=>""HVDC Ultranet"", ""ref""=>""https://tyndp...",True,False,-1.0,...,1.000000,0,inf,inf,0.0,1.0,0.0,1.0,NaN,NaN
DE0 2 oil plant,DE0 2 oil,DE0 2,oil,NaN,0.000000,199.000000,NaN,NaN,NaN,0.0,...,0.357667,0,inf,inf,0.0,1.0,125.0,1.0,NaN,NaN
DE0 2 coal plant,DE0 2 coal,DE0 2,coal,NaN,0.000000,4405.936715,NaN,NaN,NaN,0.0,...,0.454788,0,inf,inf,0.0,1.0,22.0,1.0,NaN,NaN
DE0 9 coal plant,DE0 9 coal,DE0 9,coal,NaN,0.000000,2992.929426,NaN,NaN,NaN,0.0,...,0.468952,0,inf,inf,0.0,1.0,22.0,1.0,NaN,NaN
DE0 A oil plant,DE0 A oil,DE0 A,oil,NaN,0.000000,187.200000,NaN,NaN,NaN,0.0,...,0.382780,0,inf,inf,0.0,1.0,125.0,1.0,NaN,NaN
DE0 A coal plant,DE0 A coal,DE0 A,coal,NaN,0.000000,7497.505265,NaN,NaN,NaN,0.0,...,0.444460,0,inf,inf,0.0,1.0,22.0,1.0,NaN,NaN


In [40]:
n.links_t.p1[n.links[n.links.carrier == "gas"].index].sum().sum()


0.0

In [35]:
n.links_t.p0.filter(like="biomass_boiler").sum()

Link
DE0 2 biomass_boiler    15.465273
DE0 9 biomass_boiler    12.449011
DE0 A biomass_boiler    15.474974
DE0 E biomass_boiler    12.656731
dtype: float64

In [21]:
print(n.links_t.p0.filter(like="CCGT").sum())

Link
DE0 2 CCGT    7.099439e+07
DE0 9 CCGT    6.227198e-05
DE0 A CCGT    2.055593e+08
DE0 E CCGT    5.777393e-05
dtype: float64


In [37]:
print(n.links_t.p0.filter(like="oil_boiler").sum())

Link
DE0 2 oil_boiler    8.295163
DE0 9 oil_boiler    6.544375
DE0 A oil_boiler    8.781147
DE0 E oil_boiler    6.466062
dtype: float64


In [38]:
print(n.links_t.p0.filter(like="heat_pump").sum())

Link
DE0 2 heat_pump    3.498395e+07
DE0 9 heat_pump    3.425490e+07
DE0 A heat_pump    3.460667e+07
DE0 E heat_pump    3.370413e+07
dtype: float64


In [41]:
# Summaries of heating sector elements
n_heat=n
heating_buses_summary = n_heat.buses[n_heat.buses.index.str.contains('heat')][['v_nom', 'x', 'y', 'carrier']]
heating_loads_summary = n_heat.loads[n_heat.loads.carrier == 'heat'][['bus', 'p_set', 'carrier']]
heating_links_summary = n_heat.links[n_heat.links.carrier.str.contains('boiler', case=False, na=False)][['bus0', 'bus1', 'efficiency', 'p_nom_extendable', 'carrier']]

print("Heating Buses:")
print(heating_buses_summary)
print("\nHeating Loads:")
print(heating_loads_summary)
print("\nHeating Links:")
print(heating_links_summary)

Heating Buses:
            v_nom          x          y carrier
Bus                                            
DE0 9 heat    1.0   9.690522  53.086620    heat
DE0 A heat    1.0   7.702589  50.884153    heat
DE0 2 heat    1.0   9.919611  48.832362    heat
DE0 E heat    1.0  12.855435  51.891973    heat

Heating Loads:
                               bus  p_set carrier
Load                                             
DE0 9 heat_residential  DE0 9 heat    0.0    heat
DE0 A heat_residential  DE0 A heat    0.0    heat
DE0 2 heat_residential  DE0 2 heat    0.0    heat
DE0 E heat_residential  DE0 E heat    0.0    heat

Heating Links:
                           bus0        bus1  efficiency  p_nom_extendable  \
Link                                                                        
DE0 9 oil_boiler          DE0 9  DE0 9 heat        0.90              True   
DE0 A oil_boiler          DE0 A  DE0 A heat        0.90              True   
DE0 2 oil_boiler          DE0 2  DE0 2 heat        0.90  

In [44]:
# Check heating links added
print(n.links[n.links.carrier.str.contains("heat|boiler", case=False, na=False)])

                           bus0        bus1         carrier geometry  length  \
Link                                                                           
DE0 9 oil_boiler          DE0 9  DE0 9 heat      oil_boiler              0.0   
DE0 A oil_boiler          DE0 A  DE0 A heat      oil_boiler              0.0   
DE0 2 oil_boiler          DE0 2  DE0 2 heat      oil_boiler              0.0   
DE0 E oil_boiler          DE0 E  DE0 E heat      oil_boiler              0.0   
DE0 9 heat_pump           DE0 9  DE0 9 heat       heat_pump              0.0   
DE0 A heat_pump           DE0 A  DE0 A heat       heat_pump              0.0   
DE0 2 heat_pump           DE0 2  DE0 2 heat       heat_pump              0.0   
DE0 E heat_pump           DE0 E  DE0 E heat       heat_pump              0.0   
DE0 9 biomass_boiler      DE0 9  DE0 9 heat  biomass_boiler              0.0   
DE0 A biomass_boiler      DE0 A  DE0 A heat  biomass_boiler              0.0   
DE0 2 biomass_boiler      DE0 2  DE0 2 h

In [45]:
print(n.links[n.links.carrier.isin(["oil_boiler", "heat_pump", "CCGT", "OCGT"])][["capital_cost", "marginal_cost"]])


                  capital_cost  marginal_cost
Link                                         
DE0 2 CCGT        85966.901756          4.004
DE0 2 OCGT        16893.824562          1.800
DE0 9 CCGT        85966.901756          4.004
DE0 9 OCGT        16893.824562          1.800
DE0 A CCGT        85966.901756          4.004
DE0 A OCGT        16893.824562          1.800
DE0 E CCGT        85966.901756          4.004
DE0 E OCGT        16893.824562          1.800
DE0 9 oil_boiler     98.260838         70.000
DE0 A oil_boiler     98.260838         70.000
DE0 2 oil_boiler     98.260838         70.000
DE0 E oil_boiler     98.260838         70.000
DE0 9 heat_pump      74.865400          0.000
DE0 A heat_pump      74.865400          0.000
DE0 2 heat_pump      74.865400          0.000
DE0 E heat_pump      74.865400          0.000


In [46]:
print(n.links[n.links.bus2.str.contains("heat")][["carrier", "p_nom_opt"]])


           carrier     p_nom_opt
Link                            
DE0 2 CCGT    CCGT  11742.115452
DE0 9 CCGT    CCGT      0.011990
DE0 A CCGT    CCGT  22136.839936
DE0 E CCGT    CCGT      0.009246


In [5]:
n.loads_t.p_set

Load,DE0 2,DE0 9,DE0 A,DE0 E,DE0 9 heat_residential,DE0 A heat_residential,DE0 2 heat_residential,DE0 E heat_residential
snapshot,,,,,,,,
2013-01-01 00:00:00,14530.320704,7144.829197,13240.135309,6041.463687,6035.448242,11600.076390,9637.759316,5742.965771
2013-01-01 01:00:00,13943.600418,6856.327910,12705.511460,5797.515230,3046.159128,5854.690019,4864.286353,2898.539910
2013-01-01 02:00:00,13386.236792,6582.261838,12197.637616,5565.772781,1341.819860,2578.965514,2142.697002,1276.794236
2013-01-01 03:00:00,13179.912999,6480.808588,12009.633856,5479.986808,1780.442740,3421.994682,2843.115857,1694.161113
2013-01-01 04:00:00,12952.088554,6368.783067,11802.038543,5385.261224,3913.312397,7521.350681,6249.007778,3723.670264
...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,19007.836013,9346.506831,17320.080256,7903.139467,27822.192610,53474.000050,44428.116230,26473.907720
2013-12-31 20:00:00,18419.048356,9056.988979,16783.572604,7658.331433,23137.743350,44470.531370,36947.711690,22016.470490
2013-12-31 21:00:00,18508.358833,9100.904603,16864.952969,7695.465232,23153.773950,44501.342030,36973.310290,22031.724230


In [13]:
heating_links = n1.links[n1.links.carrier.isin(["oil_boiler", "heat_pump"])]
p_by_carrier = heating_links.groupby("carrier").p_nom_opt.sum()
print(p_by_carrier)


carrier
heat_pump     93082.961788
oil_boiler        4.979288
Name: p_nom_opt, dtype: float64


In [15]:
heat_links = n.links[n.links.bus1.str.contains('heat')]
print(heat_links.carrier.value_counts())


oil_boiler    4
heat_pump     4
Name: carrier, dtype: int64


In [26]:
n.component_attrs["Link"]


,type,unit,default,description,status
attribute,,,,,
name,string,NaN,NaN,Unique name,Input (required)
bus0,string,NaN,NaN,Name of first bus to which link is attached.,Input (required)
bus1,string,NaN,NaN,Name of second bus to which link is attached.,Input (required)
type,string,NaN,NaN,Placeholder for link type. Not yet implemented.,Input (optional)
carrier,string,NaN,NaN,Energy carrier transported by the link: can be...,Input (optional)
efficiency,static or series,per unit,1,Efficiency of power transfer from bus0 to bus1...,Input (optional)
build_year,int,year,0,build year,Input (optional)
lifetime,float,years,inf,lifetime,Input (optional)
p_nom,float,MW,0,Limit of active power which can pass through l...,Input (optional)


In [27]:

# Load your network (or create a dummy one)

# Extend the component attributes for 3-bus support
if "bus2" not in n.component_attrs["Link"].index:
    n.component_attrs["Link"].loc["bus2"] = [
        "string", None, None, "Name of third bus (heat) to which link is attached", "Input (optional)"
    ]

if "efficiency2" not in n.component_attrs["Link"].index:
    n.component_attrs["Link"].loc["efficiency2"] = [
        "static or series", "per unit", 1, "Efficiency of heat output", "Input (optional)"
    ]



In [28]:

# Print and check the full list of link attributes
n.component_attrs["Link"]

,type,unit,default,description,status
attribute,,,,,
name,string,NaN,NaN,Unique name,Input (required)
bus0,string,NaN,NaN,Name of first bus to which link is attached.,Input (required)
bus1,string,NaN,NaN,Name of second bus to which link is attached.,Input (required)
type,string,NaN,NaN,Placeholder for link type. Not yet implemented.,Input (optional)
carrier,string,NaN,NaN,Energy carrier transported by the link: can be...,Input (optional)
efficiency,static or series,per unit,1,Efficiency of power transfer from bus0 to bus1...,Input (optional)
build_year,int,year,0,build year,Input (optional)
lifetime,float,years,inf,lifetime,Input (optional)
p_nom,float,MW,0,Limit of active power which can pass through l...,Input (optional)


In [ ]:
# Read in hourly demand (MW)
df = pd.read_csv("C:\\Users\\86435\\project_h2\\data\\heat_demand\\when2heat_multiindex.csv", header=[0,1,2,3], index_col=0, parse_dates=True)

# Select only MW data
mw_df = df.xs('MW', axis=1, level='unit')

# Compute energy per column (MWh) by summing MW * 1h
total_energy_MWh = mw_df.sum()

# OR, if you want a time series of total hourly heat demand (MW):
total_hourly_MW = mw_df.sum(axis=1)


C:\Users\86435\AppData\Local\Temp\ipykernel_11356\3085141207.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("C:\\Users\\86435\\project_h2\\data\\heat_demand\\when2heat_multiindex.csv", header=[0,1,2,3], index_col=0, parse_dates=True)


In [27]:
mw_df['attribute']['total'].plot()

KeyError: 'attribute'

In [17]:
# Step 1: Reload and extract MW-level data
df = pd.read_csv("C:\\Users\\86435\\project_h2\\data\\heat_demand\\when2heat_multiindex.csv", header=[0,1,2,3], index_col=0, parse_dates=True)
mw_df = df.xs('MW', axis=1, level='unit')

# Step 2: Sum MWh per category
sector_energy = mw_df.sum() / 1000  # GWh
sector_energy.index = sector_energy.index.to_frame(index=False).agg('_'.join, axis=1)

# Step 3: Sort by category
sector_energy.sort_values(ascending=False)


C:\Users\86435\AppData\Local\Temp\ipykernel_11356\3688183452.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("C:\\Users\\86435\\project_h2\\data\\heat_demand\\when2heat_multiindex.csv", header=[0,1,2,3], index_col=0, parse_dates=True)


DE_heat_demand_total        581732.937
DE_heat_demand_space        475597.348
DE_heat_demand_space_SFH    260163.204
DE_heat_demand_space_MFH    111498.337
DE_heat_demand_water        106135.427
DE_heat_demand_space_COM    103935.526
DE_heat_demand_water_SFH     55404.489
DE_heat_demand_water_COM     26986.787
DE_heat_demand_water_MFH     23745.157
dtype: float64

In [13]:
space_total = sector_energy.filter(like='space').sum()
water_total = sector_energy.filter(like='water').sum()


In [14]:
space_total

951194.4149999999

# Map germany

In [3]:
pip install cartopy

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 932.9 kB/s eta 0:00:12
   --- ------------------------------------ 1.0/11.0 MB 1.3 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.0 MB 1.5 MB/s eta 0:00:07
   ------ --------------------------------- 1.8/11.0 MB 1.7 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.0 MB 1.9 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.0 MB 1.7 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.0 MB 1.8 MB/s eta 0:00:05
   ------------- -------------------------- 3.7/11.0 MB 1.9 MB/s eta 0:00:04
   ---------------- ----------------------- 4.5/11.0 MB 2.0 MB/s eta 0:00:04
   ----------------- ---------------------- 4.7/11.0 MB 2.1 MB/s eta 0:00:03
   ---------------

In [4]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

# # Crear el dataset a partir de la imagen
buses = pd.DataFrame({
    'x': [9.056326, 11.132877, 13.420363, 8.749966, 12.924769, 8.979173, 7.425627, 7.407383, 6.917756, 13.399063, 11.935272, 9.698857, 11.486402],
    'y': [48.698691, 49.020224, 52.587492, 52.282211, 53.96037, 52.695031, 51.404921, 49.92631, 49.404465, 51.157059, 51.855145, 53.798895, 50.799502],
    'name': ['DE0 1', 'DE0 2', 'DE0 4', 'DE0 7', 'DE0 8', 'DE0 9', 'DE0 A', 'DE0 B', 'DE0 C', 'DE0 D', 'DE0 E', 'DE0 F', 'DE0 G']
})

buses = pd.DataFrame()
buses["name"] = n.buses.index.tolist()
buses["x"] = n.buses.x.to_list()
buses["y"] = n.buses.y.to_list()

# Crear figura y ejes con proyección geográfica
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Agregar fondo de mapa con bordes de países y costas
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black')
ax.add_feature(cfeature.COASTLINE, edgecolor='black')
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.set_extent([5, 15, 47, 56], crs=ccrs.PlateCarree())  # Limitar el mapa a Alemania

# Graficar buses
ax.scatter(buses['x'], buses['y'], color='red', s=50, transform=ccrs.PlateCarree(), label="Buses")

# Etiquetas
for i, row in buses.iterrows():
    ax.text(row['x'], row['y'], row['name'], fontsize=10, transform=ccrs.PlateCarree(), ha='right')

ax.set_title("Mapa de Buses en Alemania")
plt.legend()
plt.show()

NameError: name 'n' is not defined

# Code

In [1]:
import os
import logging
import pandas as pd
import yaml
import pypsa
import warnings

import Model_Code.data_processing as dp
import Model_Code.base_functions as bf
import Model_Code.Myopic as mp
import Model_Code.plotting as pt
import Model_Code.H2_Ready as H2R

from Model_Code.Constraints import extra_functionality
warnings.filterwarnings("ignore")

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

ERROR 1: PROJ: proj_create_from_database: Open of /home/pguerra/anaconda3/envs/mypypsa1/share/proj failed


In [2]:
# Read config.yaml
with open(r'config.yaml', "r") as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

opts = config['scenario_settings']['opts'][0].split('-')
solver_options = config['solving']['solver'].copy()
tmpdir = config['solving'].get('tmpdir')
colors=config['plotting']['tech_colors']

#extract scenario settings
scenario_settings = config.get("scenario_settings", {})
#print('enter clusters')
#clusters = int(input())    # default to 4 if not alligned with NUTS
clusters = 13
clusters = clusters if clusters in [4,13,37,194] else \
    scenario_settings.get("clusters")
regional_potential = scenario_settings.get("regional_potential")[clusters]
#demand_growth = scenario_settings.get("demand_growth", 1.0)
# Possibly more, e.g. co2_price or co2_limit,demand_growth (still not implemented)
logger.info("Using config-based scenario settings:")
logger.info(f" clusters={clusters}, regional_potential={regional_potential}")

# Construct a network path from the "Networks" folder
#    Example file: "Networks/elec_s_3_ec_lcopt_Co2L-1H-Ep-CCL.nc" if clusters=3
network_filename = f"Networks/elec_s_{clusters}_ec_lcopt_Co2L-1H-Ep-CCL.nc"
if not os.path.exists(network_filename):
    raise FileNotFoundError(f"Cannot find network file: {network_filename}")


# Load the PyPSA Network
n = pypsa.Network(network_filename)
#n.generators["p_nom_min"] = 0

INFO:pypsa.io:Imported network elec_s_13_ec_lcopt_Co2L-1H-Ep-CCL.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


In [4]:
n.generators.carrier.value_counts()

CCGT          13
OCGT          13
biomass       13
onwind        13
solar         13
ror           12
coal          10
oil            7
lignite        6
offwind-ac     3
offwind-dc     3
Name: carrier, dtype: int64

In [5]:
n.links.carrier.value_counts()

DC    4
Name: carrier, dtype: int64

In [6]:
n.loads_t.p_set

Load,DE0 1,DE0 2,DE0 4,DE0 7,DE0 8,DE0 9,DE0 A,DE0 B,DE0 C,DE0 D,DE0 E,DE0 F,DE0 G
snapshot,,,,,,,,,,,,,
2013-01-01 00:00:00,5614.752402,8085.696784,1939.085292,3469.759586,561.306055,4577.118930,6620.561036,1955.112825,438.516819,1350.521673,1095.824481,1598.340937,1310.985580
2013-01-01 01:00:00,5388.034134,7759.204174,1860.786904,3329.654052,538.641060,4392.299298,6353.229189,1876.167262,420.809934,1295.988915,1051.576150,1533.801477,1258.049251
2013-01-01 02:00:00,5172.659758,7449.047683,1786.406192,3196.558725,517.110112,4216.727151,6099.273342,1801.171754,403.989017,1244.184714,1009.541791,1472.491261,1207.761601
2013-01-01 03:00:00,5092.932886,7334.234551,1758.872083,3147.289753,509.139827,4151.734192,6005.264456,1773.410062,397.762282,1225.007935,993.981593,1449.795563,1189.146216
2013-01-01 04:00:00,5004.897813,7207.456938,1728.468691,3092.886546,500.338973,4079.968427,5901.459063,1742.755371,390.886670,1203.832776,976.799894,1424.734786,1168.590953
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,7344.937189,10577.302570,2536.614022,4538.965282,734.272400,5987.557178,8660.685545,2557.580437,573.645686,1766.684646,1433.502568,2090.869365,1714.965514
2013-12-31 20:00:00,7117.420056,10249.659529,2458.039742,4398.366071,711.527542,5802.086315,8392.411726,2478.356700,555.876409,1711.959736,1389.098322,2026.102493,1661.842658
2013-12-31 21:00:00,7151.931078,10299.358187,2469.958311,4419.692915,714.977604,5830.219532,8433.104941,2490.373782,558.571749,1720.260705,1395.833797,2035.926680,1669.900618


In [3]:
# Load the PyPSA Network
n = pypsa.Network(network_filename)
n.generators["p_nom_min"] = 0

# Create results folder(s)
# For example, strip the ".nc" from network file for one folder,
# and also create a folder named after the number of buses:
# name = network_filename[:-3]  # e.g. "Networks/elec_s_3_ec_lcopt_Co2L-1H-Ep-CCL"
# bf.createFolder(name)
name = "Networks/elec_s_3_ec_lcopt_Co2L-1H-Ep-CCL"

# bus_folder = 'Results/'+str(int(len(n.buses)))
# bf.createFolder(bus_folder)

# # Update renewable profiles, availability, etc.
# dp.update_rens_profiles(n, reference_year=2013, name=name)
# bf.update_availability_profiles(n, name)

INFO:pypsa.io:Imported network elec_s_13_ec_lcopt_Co2L-1H-Ep-CCL.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


In [9]:
def update_rens_profiles(n,reference_year,name):  

    """ Updates renewable energy profiles for solar and wind generators in the network.

    This function fetches data from the Renewables.ninja API for the specified year, storing the profiles 
    in 'gen_profiles.csv'. If the file exists, it updates missing generator profiles. The function includes 
    a delay between API requests to avoid rate limiting.

    Parameters:
    -----------
    n : Network
        The network object containing the generators.
    reference_year : int
        The year for which to fetch renewable energy data.
    name : str
        The directory path to save the 'gen_profiles.csv' file.

    Returns:
    --------
    None
    """
    
    token = '1cacc0c037ec8d41e4ed821bcd9d1ab673766aee'
    api_base = 'https://www.renewables.ninja/api/'
    
    s = requests.session()
    s.headers = {'Authorization': 'Token ' + token}
    
    if os.path.exists('{}/gen_profiles.csv'.format(name)):
        gen_profiles=pd.read_csv('{}/gen_profiles.csv'.format(name),index_col=0)
        gen_profiles.index =n.generators_t.p_max_pu.index
    else:
        gen_profiles= pd.DataFrame(index =n.generators_t.p_max_pu.index,
                                   columns=list(n.generators.index[(n.generators.carrier=='solar') |
                                                                   (n.generators.carrier=='onwind')]))
    args_wind = {
        'date_from': str(reference_year)+'-01-01',
        'date_to': str(reference_year)+'-12-31',
        'capacity': 1.0,
        'height': 101.5, # min:84, max:119, avg:101.5
        'turbine': 'Vestas V112 3000',
        'format': 'json'}
    args_solar = {
        'date_from': str(reference_year)+'-01-01',
        'date_to': str(reference_year)+'-12-31',
        'dataset': 'merra2',
        'capacity': 1.0,
        'system_loss': 0.1,
        'tracking': 0,
        'tilt': 35,
        'azim': 180,
        'format': 'json'
        }
    args = {'solar':args_solar, 'onwind':args_wind}
    urls= {'solar':'pv', 'onwind':'wind'}
    for tech in ['solar','onwind']:
        for i in n.generators.index[n.generators.carrier==tech]:
            if (i not in gen_profiles.columns) | (math.isnan(gen_profiles[i][0])):
                args[tech]['lat']=n.buses.loc[n.generators.loc[i,'bus'],'y']
                args[tech]['lon']=n.buses.loc[n.generators.loc[i,'bus'],'x']
                url = api_base + 'data/{}'.format(urls[tech])
                r = s.get(url, params=args[tech])
                parsed_response = json.loads(r.text)
                data = pd.read_json(json.dumps(parsed_response['data']), orient='index')
                resample_factor=int(8760/len(n.snapshots))
                gen_profiles[i]=data.electricity.resample('{}H'.format(resample_factor)).mean()
                gen_profiles.to_csv('{}/gen_profiles.csv'.format(name))
                time.sleep(15)

In [10]:
def update_availability_profiles(n,name):
    """
    Updates generator availability profiles using data from a CSV file.

    Parameters:
    -----------
    n : Network
        The network object to update.
    name : str
        The directory path containing the 'gen_profiles.csv' file.

    Returns:
    --------
    None
    """
    gen_profiles=pd.read_csv('{}/gen_profiles.csv'.format(name),index_col=0)
    for gen in gen_profiles.columns:
        logger.info("Correcting {} generation profile: {} CF instead of {}"
                    .format(gen,round(gen_profiles[gen].mean(),3),
                            round(n.generators_t.p_max_pu[gen].mean(),3)))
        n.generators_t.p_max_pu[gen]=gen_profiles[gen].values

In [11]:
name = "Networks/elec_s_3_ec_lcopt_Co2L-1H-Ep-CCL"
gen_profiles=pd.read_csv('/home/pguerra/Model_Split_v3/Networks/elec_s_13_ec_lcopt_Co2L-1H-Ep-CCL/gen_profiles.csv',index_col=0)
gen_profiles


,DE0 1 onwind,DE0 1 solar,DE0 2 onwind,DE0 2 solar,DE0 4 onwind,DE0 4 solar,DE0 7 onwind,DE0 7 solar,DE0 8 onwind,DE0 8 solar,...,DE0 C onwind,DE0 C solar,DE0 D onwind,DE0 D solar,DE0 E onwind,DE0 E solar,DE0 F onwind,DE0 F solar,DE0 G onwind,DE0 G solar
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,0.508,0.0,0.400,0.0,0.591,0.0,0.463,0.0,0.768,0.0,...,0.479,0.0,0.616,0.0,0.549,0.0,0.893,0.0,0.594,0.0
2013-01-01 01:00:00,0.509,0.0,0.383,0.0,0.600,0.0,0.455,0.0,0.761,0.0,...,0.516,0.0,0.628,0.0,0.552,0.0,0.853,0.0,0.591,0.0
2013-01-01 02:00:00,0.513,0.0,0.374,0.0,0.606,0.0,0.468,0.0,0.754,0.0,...,0.543,0.0,0.637,0.0,0.545,0.0,0.808,0.0,0.600,0.0
2013-01-01 03:00:00,0.511,0.0,0.372,0.0,0.593,0.0,0.492,0.0,0.730,0.0,...,0.563,0.0,0.639,0.0,0.533,0.0,0.769,0.0,0.612,0.0
2013-01-01 04:00:00,0.505,0.0,0.387,0.0,0.577,0.0,0.506,0.0,0.707,0.0,...,0.609,0.0,0.637,0.0,0.536,0.0,0.733,0.0,0.628,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,0.361,0.0,0.222,0.0,0.314,0.0,0.366,0.0,0.390,0.0,...,0.412,0.0,0.365,0.0,0.400,0.0,0.624,0.0,0.348,0.0
2013-12-31 20:00:00,0.342,0.0,0.187,0.0,0.255,0.0,0.349,0.0,0.344,0.0,...,0.377,0.0,0.351,0.0,0.368,0.0,0.582,0.0,0.345,0.0
2013-12-31 21:00:00,0.330,0.0,0.165,0.0,0.212,0.0,0.323,0.0,0.352,0.0,...,0.360,0.0,0.344,0.0,0.324,0.0,0.540,0.0,0.344,0.0


In [12]:
n.generators_t.p_max_pu

Generator,DE0 1 onwind,DE0 1 ror,DE0 1 solar,DE0 2 onwind,DE0 2 ror,DE0 2 solar,DE0 4 onwind,DE0 4 ror,DE0 4 solar,DE0 7 onwind,...,DE0 E ror,DE0 E solar,DE0 F offwind-ac,DE0 F offwind-dc,DE0 F onwind,DE0 F ror,DE0 F solar,DE0 G onwind,DE0 G ror,DE0 G solar
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,0.292079,0.840122,0.0,0.226119,0.840122,0.0,0.831170,0.840122,0.0,0.689216,...,0.840122,0.0,0.885500,0.801092,0.996022,0.840122,0.0,0.752372,0.840122,0.0
2013-01-01 01:00:00,0.277778,0.841015,0.0,0.217793,0.841015,0.0,0.804937,0.841015,0.0,0.688697,...,0.841015,0.0,0.885296,0.741052,0.993543,0.841015,0.0,0.790577,0.841015,0.0
2013-01-01 02:00:00,0.276878,0.842234,0.0,0.213140,0.842234,0.0,0.775983,0.842234,0.0,0.664808,...,0.842234,0.0,0.876822,0.648269,0.957117,0.842234,0.0,0.789038,0.842234,0.0
2013-01-01 03:00:00,0.328744,0.843261,0.0,0.220567,0.843261,0.0,0.781507,0.843261,0.0,0.696576,...,0.843261,0.0,0.869133,0.626918,0.931495,0.843261,0.0,0.784929,0.843261,0.0
2013-01-01 04:00:00,0.352065,0.844720,0.0,0.223366,0.844720,0.0,0.746016,0.844720,0.0,0.693689,...,0.844720,0.0,0.846998,0.542550,0.864469,0.844720,0.0,0.753949,0.844720,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,0.133344,0.819189,0.0,0.127172,0.819189,0.0,0.339777,0.819189,0.0,0.350164,...,0.819189,0.0,0.659751,0.885500,0.668547,0.819189,0.0,0.318510,0.819189,0.0
2013-12-31 20:00:00,0.121719,0.819306,0.0,0.118201,0.819306,0.0,0.318776,0.819306,0.0,0.348510,...,0.819306,0.0,0.694474,0.885500,0.701080,0.819306,0.0,0.344234,0.819306,0.0
2013-12-31 21:00:00,0.113141,0.819363,0.0,0.108492,0.819363,0.0,0.303500,0.819363,0.0,0.350767,...,0.819363,0.0,0.713921,0.885500,0.697771,0.819363,0.0,0.369410,0.819363,0.0


## Datasets

In [ ]:
# Read input data / CSVs from inside the
co2lims = pd.read_csv(os.path.join("data", "co2limits.csv"))
cost_factors = pd.read_csv(os.path.join("data", "Cost_Factor.csv"), index_col=0, header=0)
fuel_cost = pd.read_csv(os.path.join("data", "fuel_cost.csv"), index_col=0, header=0)
co2price = pd.read_csv(os.path.join("data", "co2_price.csv"), index_col=0)

In [ ]:
co2lims

,year,co2limit
0,2019,250000000.0
1,2020,221000000.0
2,2021,215000000.0
3,2022,209000000.0
4,2023,203000000.0
5,2024,197000000.0
6,2025,191000000.0
7,2026,185000000.0
8,2027,179000000.0
9,2028,173000000.0


In [ ]:
cost_factors #pareciera que es en porcentaje entonces va bajando el costo de las tecnologias con los anos. 

,solar,onwind,offwind-ac,offwind-dc,battery,H2,electrolysis,fuel cell
year,,,,,,,,
2020,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2021,0.983618,0.987726,0.968284,0.968284,0.940018,0.972588,0.972588,0.972588
2022,0.983603,0.987847,0.970556,0.970556,0.942137,0.972347,0.972347,0.972347
2023,0.983586,0.987965,0.972582,0.972582,0.944052,0.972093,0.972093,0.972093
2024,0.983567,0.988081,0.974397,0.974397,0.945789,0.971823,0.971823,0.971823
2025,0.983546,0.988194,0.976030,0.976030,0.947368,0.971539,0.971539,0.971539
2026,0.983522,0.988305,0.977507,0.977507,0.948808,0.971238,0.971238,0.971238
2027,0.983496,0.988413,0.978846,0.978846,0.950124,0.970920,0.970920,0.970920
2028,0.983468,0.988519,0.980065,0.980065,0.951329,0.970584,0.970584,0.970584


In [ ]:
fuel_cost

,gas,biomass,coal,lignite,oil,H2
year,,,,,,
2019,31.50,26.380,8.730,3.20,46.40,172.845325
2020,31.50,26.380,8.730,3.20,46.40,172.845325
2021,32.34,26.528,8.864,3.24,48.98,168.323123
2022,33.18,26.676,8.998,3.28,51.56,163.800920
2023,34.02,26.824,9.132,3.32,54.14,159.278718
2024,34.86,26.972,9.266,3.36,56.72,154.756515
2025,35.70,27.120,9.400,3.40,59.30,150.234312
2026,36.52,27.268,9.540,3.44,61.90,145.712110
2027,37.34,27.416,9.680,3.48,64.50,141.189907


In [ ]:
co2price

,co2limit
year,
2019,0
2020,75
2021,75
2022,75
2023,75
2024,75
2025,75
2026,75
2027,75


## seguir

In [4]:
# Read input data / CSVs from inside the
co2lims = pd.read_csv(os.path.join("data", "co2limits.csv"))
cost_factors = pd.read_csv(os.path.join("data", "Cost_Factor.csv"), index_col=0, header=0)
fuel_cost = pd.read_csv(os.path.join("data", "fuel_cost.csv"), index_col=0, header=0)
co2price = pd.read_csv(os.path.join("data", "co2_price.csv"), index_col=0)

In [5]:
# Additional data to store scenario settings
agg_p_nom_minmax = pd.read_csv(config['scenario_settings']\
                                .get('agg_p_nom_limits'), index_col=1)  #limites por tecnologia
co2030 = (1 - (co2lims[co2lims.year == 2030].co2limit.values[0] / (460 * 10**6))) * 100
co2040 = (1 - (co2lims[co2lims.year == 2040].co2limit.values[0] / (460 * 10**6))) * 100
co2045 = (1 - (co2lims[co2lims.year == 2045].co2limit.values[0] / (460 * 10**6))) * 100
co2050 = (1 - (co2lims[co2lims.year == 2050].co2limit.values[0] / (460 * 10**6))) * 100

In [16]:
config['scenario_settings'].get('agg_p_nom_limits')

'data/agg_p_nom_minmax.csv'

In [6]:
agg_p_nom_minmax = pd.read_csv(config['scenario_settings']\
                                .get('agg_p_nom_limits'), index_col=1)
agg_p_nom_minmax

,country,min,max
carrier,,,
onwind,DE,NaN,50000
offwind-ac,DE,NaN,50000
solar,DE,NaN,50000
offwind-dc,DE,NaN,50000
battery,DE,NaN,40000
CCGT,DE,NaN,10000
OCGT,DE,NaN,14600
electrolysis,DE,NaN,62000
cell,DE,NaN,85000


In [18]:
co2030, co2040, co2045, co2050 #otros valores que el original

(65.0, 88.0, 100.0, 100.0)

In [7]:
var = [
    "regional potential",
    "CO2-reduction-2030",
    "CO2-reduction-2040",
    "CO2-reduction-2045",
    "CO2-reduction-2050",
]
var.extend(list(agg_p_nom_minmax.index))

var
val = [regional_potential, co2030, co2040, co2045, co2050]
val.extend(list(agg_p_nom_minmax['max']))

In [ ]:
var, val

(['regional potential',
  'CO2-reduction-2030',
  'CO2-reduction-2040',
  'CO2-reduction-2045',
  'CO2-reduction-2050',
  'onwind',
  'offwind-ac',
  'solar',
  'offwind-dc',
  'battery',
  'CCGT',
  'OCGT',
  'electrolysis',
  'cell',
  'H2_input'],
 [8000,
  65.0,
  88.0,
  100.0,
  100.0,
  50000,
  50000,
  50000,
  50000,
  40000,
  10000,
  14600,
  62000,
  85000,
  6000])

In [8]:
txt = pd.DataFrame({"val": val}, index=var)
txt

,val
regional potential,8000.0
CO2-reduction-2030,65.0
CO2-reduction-2040,88.0
CO2-reduction-2045,100.0
CO2-reduction-2050,100.0
onwind,50000.0
offwind-ac,50000.0
solar,50000.0
offwind-dc,50000.0
battery,40000.0


## Res Data

In [ ]:
def RES_data(n,name):
    """
    Corrects renewable energy sources (RES) data and updates the network.

    This function processes CSV files with German renewable power plant data, correcting and classifying
    technologies, and mapping plants to the nearest bus in the network. It handles offshore wind data
    separately, differentiating between AC and DC connections. The function generates and saves data
    on new plant additions and those scheduled for removal, focusing on the period between 1995 and 2018.

    Parameters:
    -----------
    n : Network
        The network object to be updated with the corrected RES data.
    
    name : str
        The directory path where the addition and removal data will be saved.

    Returns:
    --------
    tuple of pd.DataFrame
        Returns two DataFrames: one for the added capacities and one for the removed capacities.
    """
        
    data=pd.read_csv('data/renewable_power_plants_DE.csv')
    data=data[['commissioning_date','decommissioning_date','technology','electrical_capacity','lat','lon']]
    data.columns=['year_added','year_removed','carrier','p_nom','latitude','longitude']
    data.carrier=data.carrier.replace('Photovoltaics', 'solar')
    data.carrier=data.carrier.replace('Onshore', 'onwind')
    data.carrier=data.carrier.replace('Photovoltaics ground', 'solar')
    data.carrier=data.carrier.replace('Run-of-river', 'ror')
    data.carrier=data.carrier.replace('Sewage and landfill gas', 'biomass')
    data.carrier=data.carrier.replace('Biomass and biogas', 'biomass')
    data.drop(data[data.carrier == 'biomass' ].index, inplace=True)

    """
    Change Offshore data
    """
    a=data[data.carrier=='Offshore']
    vals=pd.DataFrame({'lat_vals':a.latitude.unique()})
    vals.drop(vals[np.isnan(vals.lat_vals) ==True].index, inplace=True)
    a.latitude[np.isnan(a.latitude)]=vals.mean()[0]
    
    vals=pd.DataFrame({'long_vals':a.longitude.unique()})
    vals.drop(vals[np.isnan(vals.long_vals) ==True].index, inplace=True)
    
    a.longitude[np.isnan(a.longitude)] = np.random.choice(vals.long_vals,
                size=len(a.longitude[np.isnan(a.longitude)]))
    j=0
    for i in a.index:
        if a.carrier[i] == 'Offshore':
            j=j+1
            if j % 2 ==0:
                a.carrier[i]=('offwind-ac')
            else:
                a.carrier[i]=('offwind-dc')
    
    for i in a.index:
        data.loc[i,'carrier']=a.loc[i,'carrier']
        data.loc[i,'latitude']=a.loc[i,'latitude']
        data.loc[i,'longitude']=a.loc[i,'longitude']

    """ """

    data.index=range(len(data))
    
    year_added=[]
    year_removed=[]
    for i in range(len(data)):
        year_added.append( int(data[data.columns[0]][i][:4]) )
        year_removed.append( int(data[data.columns[0]][i][:4]) +25)
    
    data.year_added=year_added
    data.year_removed=year_removed
    
    data.drop(data[data.year_added < 1995 ].index, inplace=True)
    data.drop(data[data.year_added > 2018].index, inplace=True)
    data.drop(data[data.carrier == 'Other fossil fuels' ].index, inplace=True)
    data.drop(data[data.carrier == 'Geothermal' ].index, inplace=True)
    data.drop(data[data.carrier == 'Storage' ].index, inplace=True)
    
    
    data.drop(data[np.isnan(data.longitude) ==True].index, inplace=True)
    data.drop(data[np.isnan(data.latitude) ==True].index, inplace=True)
    data.drop(data[np.isnan(data.p_nom) ==True].index, inplace=True)
    

    
    data.drop(data[data.carrier=='ror'].index, inplace=True)
    data.drop(data[np.isnan(data.longitude) ==True].index, inplace=True)
    data.drop(data[np.isnan(data.latitude) ==True].index, inplace=True)
    data.index=range(len(data))

    ### 2019 data
    df2=pd.read_csv('data/renewable_power_plants_DE_2019.csv')
    df2=df2[['commissioning_date','decommissioning_date','technology','electrical_capacity','lat','lon']]
    df2.columns=['year_added','year_removed','carrier','p_nom','latitude','longitude']
    df2.carrier=df2.carrier.replace('Photovoltaics', 'solar')
    df2.carrier=df2.carrier.replace('Onshore', 'onwind')
    df2.carrier=df2.carrier.replace('Photovoltaics ground', 'solar')
    df2.carrier=df2.carrier.replace('Run-of-river', 'ror')
    df2.carrier=df2.carrier.replace('Sewage and landfill gas', 'biomass')
    df2.carrier=df2.carrier.replace('Biomass and biogas', 'biomass')
    df2.drop(df2[df2.carrier == 'biomass' ].index, inplace=True)

    # Correct offiwnd
    a_19=df2[df2.carrier=='Offshore']
    vals=pd.DataFrame({'lat_vals':a.latitude.unique()})
    vals.drop(vals[np.isnan(vals.lat_vals) ==True].index, inplace=True)
    a_19.latitude[np.isnan(a_19.latitude)]=vals.mean()[0]
    
    vals=pd.DataFrame({'long_vals':a.longitude.unique()})
    vals.drop(vals[np.isnan(vals.long_vals) ==True].index, inplace=True)
    
    a_19.longitude[np.isnan(a_19.longitude)] = np.random.choice(vals.long_vals,
                size=len(a_19.longitude[np.isnan(a_19.longitude)]))
    j=0
    for i in a_19.index:
        if a_19.carrier[i] == 'Offshore':
            j=j+1
            if j % 2 ==0:
                a_19.carrier[i]=('offwind-ac')
            else:
                a_19.carrier[i]=('offwind-dc')
    
    for i in a_19.index:
        df2.loc[i,'carrier']=a_19.loc[i,'carrier']
        df2.loc[i,'latitude']=a_19.loc[i,'latitude']
        df2.loc[i,'longitude']=a_19.loc[i,'longitude']
        
    
    df2.drop(df2[np.isnan(df2.longitude) ==True].index, inplace=True)
    df2.drop(df2[np.isnan(df2.latitude) ==True].index, inplace=True)
    df2.drop(df2[np.isnan(df2.p_nom) ==True].index, inplace=True)
    df2.drop(df2[df2.carrier == 'ror' ].index, inplace=True)
    df2.drop(df2[df2.carrier == 'Geothermal' ].index, inplace=True)
    
    
    df2.index=range(len(df2))
    
    year_added=[]
    year_removed=[]
    for i in range(len(df2)):
        year_added.append( int(df2[df2.columns[0]][i][:4]) )
        year_removed.append( int(df2[df2.columns[0]][i][:4]) +25)

    df2.year_added=year_added
    df2.year_removed=year_removed    
    
    data=pd.concat([data,df2])
    data.index=range(len(data))
    
    
    """" Mapping """
    data['bus']=data.latitude*0
    a=n.buses[['x','y']]
    a['bus']=a.index
    a.index=range(len(a))
    a.y*=pi/180
    a.x*=pi/180
    data.longitude*=pi/180
    data.latitude*=pi/180
    """"""
    
    data['bus'] = ut.calculate_nearest_bus(data, a, n)
    
    addition=data[['carrier','p_nom','bus']]
    addition.index=addition.bus + ' ' + addition.carrier
    add=addition.set_index(['carrier','bus']).stack().reset_index()
    add.columns=['carrier','bus','remove','p_nom']
    add=add.groupby(['carrier','bus']).agg({'p_nom':['sum']})
    bus=[]
    carrier=[]
    idx=[]
    for i in add.index:
        carrier.append(i[0])
        bus.append(i[1])
        idx.append(i[1] + ' ' + i[0])
    
    add['carrier']=carrier
    add['bus']=bus
    add.index=idx
    add.columns=['p_nom','carrier','bus']
    add.to_csv('{}/res_basic_addition.csv'.format(name))
    
    #TODO : All in one, save time
    removal=data[['year_removed','carrier','p_nom','bus']]
    years_res=list(removal.year_removed.value_counts().index)
    years_res.sort()
    remove= pd.DataFrame(columns = removal.columns)    
    for i in years_res:
        dat=removal.loc[removal.year_removed == i]
    
        df = dat.groupby(['year_removed', 'bus','carrier']).sum().sum(
            level=['year_removed', 'bus','carrier']).unstack('year_removed').fillna(0).reset_index()
    
        df['year']=[df.columns[2][1]]*len(df)
        df.columns=['bus','carrier','p_nom','year_removed']
        remove= pd.concat([df, remove], ignore_index=False)
    
    remove.index= remove.bus + ' ' + remove.carrier
    remove.to_csv('{}/res_basic_removal.csv'.format(name))

    return add,remove


In [24]:
data=pd.read_csv('data/renewable_power_plants_DE.csv')
data=data[['commissioning_date','decommissioning_date','technology','electrical_capacity','lat','lon']]
# data.columns=['year_added','year_removed','carrier','p_nom','latitude','longitude']
# data.carrier=data.carrier.replace('Photovoltaics', 'solar')
# data.carrier=data.carrier.replace('Onshore', 'onwind')
# data.carrier=data.carrier.replace('Photovoltaics ground', 'solar')
# data.carrier=data.carrier.replace('Run-of-river', 'ror')
# data.carrier=data.carrier.replace('Sewage and landfill gas', 'biomass')
# data.carrier=data.carrier.replace('Biomass and biogas', 'biomass')
# data.drop(data[data.carrier == 'biomass' ].index, inplace=True)

In [25]:
data

,commissioning_date,decommissioning_date,technology,electrical_capacity,lat,lon
0,1901-01-01,NaN,Run-of-river,0.12000,47.846131,9.664273
1,1901-01-01,NaN,Run-of-river,0.05500,47.846131,9.664273
2,1901-07-01,NaN,Run-of-river,0.04400,49.355539,8.775571
3,1901-11-01,NaN,Run-of-river,0.08000,49.355539,8.775571
4,1904-01-01,NaN,Run-of-river,0.05000,49.162694,9.853297
...,...,...,...,...,...,...
1805907,2019-01-11,NaN,Photovoltaics,0.00672,50.681730,6.493388
1805908,2019-01-11,NaN,Photovoltaics,0.00930,48.218401,12.140532
1805909,2019-01-11,NaN,Photovoltaics,0.00679,51.912080,12.276554
1805910,2019-01-14,NaN,Photovoltaics,0.00990,48.701869,11.815692


In [29]:
data.technology.value_counts()

Photovoltaics              1753870
Onshore                      28238
Biomass and biogas           14326
Run-of-river                  7134
Offshore                      1305
Sewage and landfill gas        547
Photovoltaics ground           405
Other fossil fuels              75
Geothermal                      11
Storage                          1
Name: technology, dtype: int64

In [30]:
data.columns=['year_added','year_removed','carrier','p_nom','latitude','longitude']
data.carrier=data.carrier.replace('Photovoltaics', 'solar')
data.carrier=data.carrier.replace('Onshore', 'onwind')
data.carrier=data.carrier.replace('Photovoltaics ground', 'solar')
data.carrier=data.carrier.replace('Run-of-river', 'ror')
data.carrier=data.carrier.replace('Sewage and landfill gas', 'biomass')
data.carrier=data.carrier.replace('Biomass and biogas', 'biomass')
data.drop(data[data.carrier == 'biomass' ].index, inplace=True)
data

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,1901-01-01,NaN,ror,0.12000,47.846131,9.664273
1,1901-01-01,NaN,ror,0.05500,47.846131,9.664273
2,1901-07-01,NaN,ror,0.04400,49.355539,8.775571
3,1901-11-01,NaN,ror,0.08000,49.355539,8.775571
4,1904-01-01,NaN,ror,0.05000,49.162694,9.853297
...,...,...,...,...,...,...
1805907,2019-01-11,NaN,solar,0.00672,50.681730,6.493388
1805908,2019-01-11,NaN,solar,0.00930,48.218401,12.140532
1805909,2019-01-11,NaN,solar,0.00679,51.912080,12.276554
1805910,2019-01-14,NaN,solar,0.00990,48.701869,11.815692


In [ ]:
"""
Change Offshore data
"""
import numpy as np
#Sustituye los valores NaN en a.latitude con la media de los valores disponibles. lo mismo para longitud
# clasifica la mitad en ac y la otra en dc
#actualiza el dataset
#demora harto por los for, se podria usar vectorizacion de pandas y numpy

a=data[data.carrier=='Offshore']
vals=pd.DataFrame({'lat_vals':a.latitude.unique()})
vals.drop(vals[np.isnan(vals.lat_vals) ==True].index, inplace=True)
a.latitude[np.isnan(a.latitude)]=vals.mean()[0]

vals=pd.DataFrame({'long_vals':a.longitude.unique()})
vals.drop(vals[np.isnan(vals.long_vals) ==True].index, inplace=True)

a.longitude[np.isnan(a.longitude)] = np.random.choice(vals.long_vals,
            size=len(a.longitude[np.isnan(a.longitude)]))
j=0
for i in a.index:
    if a.carrier[i] == 'Offshore':
        j=j+1
        if j % 2 ==0:
            a.carrier[i]=('offwind-ac')
        else:
            a.carrier[i]=('offwind-dc')

for i in a.index:
    data.loc[i,'carrier']=a.loc[i,'carrier']
    data.loc[i,'latitude']=a.loc[i,'latitude']
    data.loc[i,'longitude']=a.loc[i,'longitude']

In [36]:
data

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,1901-01-01,NaN,ror,0.12000,47.846131,9.664273
1,1901-01-01,NaN,ror,0.05500,47.846131,9.664273
2,1901-07-01,NaN,ror,0.04400,49.355539,8.775571
3,1901-11-01,NaN,ror,0.08000,49.355539,8.775571
4,1904-01-01,NaN,ror,0.05000,49.162694,9.853297
...,...,...,...,...,...,...
1805907,2019-01-11,NaN,solar,0.00672,50.681730,6.493388
1805908,2019-01-11,NaN,solar,0.00930,48.218401,12.140532
1805909,2019-01-11,NaN,solar,0.00679,51.912080,12.276554
1805910,2019-01-14,NaN,solar,0.00990,48.701869,11.815692


In [37]:
data.index=range(len(data))

In [38]:
data

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,1901-01-01,NaN,ror,0.12000,47.846131,9.664273
1,1901-01-01,NaN,ror,0.05500,47.846131,9.664273
2,1901-07-01,NaN,ror,0.04400,49.355539,8.775571
3,1901-11-01,NaN,ror,0.08000,49.355539,8.775571
4,1904-01-01,NaN,ror,0.05000,49.162694,9.853297
...,...,...,...,...,...,...
1791034,2019-01-11,NaN,solar,0.00672,50.681730,6.493388
1791035,2019-01-11,NaN,solar,0.00930,48.218401,12.140532
1791036,2019-01-11,NaN,solar,0.00679,51.912080,12.276554
1791037,2019-01-14,NaN,solar,0.00990,48.701869,11.815692


In [39]:
i=5
int(data[data.columns[0]][i][:4])

1905

In [ ]:
year_added=[] #14.4 s
year_removed=[]
for i in range(len(data)):
    year_added.append( int(data[data.columns[0]][i][:4]) )
    year_removed.append( int(data[data.columns[0]][i][:4]) +25)


In [ ]:
# Resetear el índice  #0.3 s
data.reset_index(drop=True, inplace=True)

# Extraer los años de manera vectorizada
data['year_added'] = data.iloc[:, 0].str[:4].astype(int)
data['year_removed'] = data['year_added'] + 25


In [44]:
data.carrier.value_counts()

solar                 1754275
onwind                  28238
ror                      7134
offwind-dc                653
offwind-ac                652
Other fossil fuels         75
Geothermal                 11
Storage                     1
Name: carrier, dtype: int64

In [45]:
data.drop(data[data.year_added < 1995].index, inplace=True)
data.drop(data[data.year_added > 2018].index, inplace=True)


In [46]:
data

,year_added,year_removed,carrier,p_nom,latitude,longitude
4626,1995,2020,solar,0.00129,49.277654,7.036675
4627,1995,2020,ror,0.01100,48.963994,10.881562
4628,1995,2020,solar,0.00880,52.651042,13.708159
4629,1995,2020,ror,0.02200,49.720897,10.825612
4630,1995,2020,solar,0.00260,50.742935,11.342881
...,...,...,...,...,...,...
1790720,2018,2043,solar,0.01220,51.217980,6.270472
1790721,2018,2043,solar,0.00180,53.650304,12.313511
1790722,2018,2043,solar,0.01980,49.522844,10.206178
1790723,2018,2043,solar,0.00976,52.293092,13.694022


In [47]:
data.drop(data[data.carrier == 'Other fossil fuels'].index, inplace=True)
data.drop(data[data.carrier == 'Geothermal'].index, inplace=True)
data.drop(data[data.carrier == 'Storage'].index, inplace=True)


In [48]:
data.drop(data[np.isnan(data.longitude)].index, inplace=True)
data.drop(data[np.isnan(data.latitude)].index, inplace=True)
data.drop(data[np.isnan(data.p_nom)].index, inplace=True)


In [49]:
data.drop(data[data.carrier=='ror'].index, inplace=True)
data.drop(data[np.isnan(data.longitude) ==True].index, inplace=True)
data.drop(data[np.isnan(data.latitude) ==True].index, inplace=True)
data.index=range(len(data))
data

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,1995,2020,solar,0.00129,49.277654,7.036675
1,1995,2020,solar,0.00880,52.651042,13.708159
2,1995,2020,solar,0.00260,50.742935,11.342881
3,1995,2020,onwind,0.08000,53.127583,12.132579
4,1995,2020,solar,0.00396,48.793771,9.528908
...,...,...,...,...,...,...
1780229,2018,2043,solar,0.01220,51.217980,6.270472
1780230,2018,2043,solar,0.00180,53.650304,12.313511
1780231,2018,2043,solar,0.01980,49.522844,10.206178
1780232,2018,2043,solar,0.00976,52.293092,13.694022


In [50]:
data.carrier.value_counts()

solar         1751441
onwind          27488
offwind-dc        653
offwind-ac        652
Name: carrier, dtype: int64

In [51]:
df2=pd.read_csv('data/renewable_power_plants_DE_2019.csv')
df2

,electrical_capacity,energy_source_level_1,energy_source_level_2,energy_source_level_3,technology,data_source,nuts_1_region,nuts_2_region,nuts_3_region,lon,...,postcode,address,federal_state,commissioning_date,decommissioning_date,voltage_level,eeg_id,dso,dso_id,tso
0,0.086940,Renewable energy,Solar,NaN,Photovoltaics,BNetzA_PV,DE2,DE26,DE268,10.332874,...,97346.0,Dr.-Ruppert-Schneider-Straße 5,Bayern,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,0.006000,Renewable energy,Solar,NaN,Photovoltaics,50Hertz,DED,DED4,DED43,13.043342,...,4736.0,NaN,Sachsen,2019-01-01,NaN,low voltage,NaN,Mitteldeutsche Netzgesellschaft Strom mbH,10001645.0,50Hertz
2,0.007470,Renewable energy,Solar,NaN,Photovoltaics,BNetzA_PV,DE2,DE23,DE235,12.391779,...,93194.0,NaN,Bayern,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,0.009720,Renewable energy,Solar,NaN,Photovoltaics,BNetzA_PV,DE1,DE11,DE11B,9.574454,...,97953.0,NaN,Baden-Württemberg,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN
4,0.009990,Renewable energy,Solar,NaN,Photovoltaics,BNetzA_PV,DE1,DE12,DE127,9.480866,...,74736.0,NaN,Baden-Württemberg,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92574,0.009920,Renewable energy,Solar,NaN,Photovoltaics,TransnetBW,DE1,DE11,DE11D,10.113042,...,73433.0,NaN,Baden-Württemberg,2019-12-31,NaN,low voltage,NaN,Stadtwerke Aalen GmbH,10001635.0,TransnetBW
92575,0.009750,Renewable energy,Solar,NaN,Photovoltaics,TenneT,DE2,DE26,DE26C,10.019952,...,97230.0,NaN,Brandenburg,2019-12-31,NaN,low voltage,NaN,Mainfranken Netze GmbH,10003071.0,TenneT
92576,0.006500,Renewable energy,Solar,NaN,Photovoltaics,TenneT,DE7,DE73,DE736,9.109868,...,34471.0,NaN,Hamburg,2019-12-31,NaN,low voltage,NaN,Energie Waldeck-Frankenberg GmbH,10000523.0,TenneT
92577,0.009800,Renewable energy,Solar,NaN,Photovoltaics,TenneT,DE7,DE71,DE719,9.382804,...,63628.0,NaN,Hamburg,2019-12-31,NaN,low voltage,NaN,OsthessenNetz GmbH,10001747.0,TenneT


In [52]:
df2=df2[['commissioning_date','decommissioning_date','technology','electrical_capacity','lat','lon']]
df2.columns=['year_added','year_removed','carrier','p_nom','latitude','longitude']
df2.carrier=df2.carrier.replace('Photovoltaics', 'solar')
df2.carrier=df2.carrier.replace('Onshore', 'onwind')
df2.carrier=df2.carrier.replace('Photovoltaics ground', 'solar')
df2.carrier=df2.carrier.replace('Run-of-river', 'ror')
df2.carrier=df2.carrier.replace('Sewage and landfill gas', 'biomass')
df2.carrier=df2.carrier.replace('Biomass and biogas', 'biomass')
df2.drop(df2[df2.carrier == 'biomass' ].index, inplace=True)

In [53]:
df2

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,2019-01-01,NaN,solar,0.086940,49.655942,10.332874
1,2019-01-01,NaN,solar,0.006000,51.072527,13.043342
2,2019-01-01,NaN,solar,0.007470,49.198889,12.391779
3,2019-01-01,NaN,solar,0.009720,49.598032,9.574454
4,2019-01-01,NaN,solar,0.009990,49.613507,9.480866
...,...,...,...,...,...,...
92574,2019-12-31,NaN,solar,0.009920,48.867734,10.113042
92575,2019-12-31,NaN,solar,0.009750,49.821914,10.019952
92576,2019-12-31,NaN,solar,0.006500,51.410735,9.109868
92577,2019-12-31,NaN,solar,0.009800,50.275753,9.382804


In [54]:
# Correct offiwnd
a_19=df2[df2.carrier=='Offshore']
vals=pd.DataFrame({'lat_vals':a.latitude.unique()})
vals.drop(vals[np.isnan(vals.lat_vals) ==True].index, inplace=True)
a_19.latitude[np.isnan(a_19.latitude)]=vals.mean()[0]

vals=pd.DataFrame({'long_vals':a.longitude.unique()})
vals.drop(vals[np.isnan(vals.long_vals) ==True].index, inplace=True)

a_19.longitude[np.isnan(a_19.longitude)] = np.random.choice(vals.long_vals,
            size=len(a_19.longitude[np.isnan(a_19.longitude)]))
j=0
for i in a_19.index:
    if a_19.carrier[i] == 'Offshore':
        j=j+1
        if j % 2 ==0:
            a_19.carrier[i]=('offwind-ac')
        else:
            a_19.carrier[i]=('offwind-dc')

for i in a_19.index:
    df2.loc[i,'carrier']=a_19.loc[i,'carrier']
    df2.loc[i,'latitude']=a_19.loc[i,'latitude']
    df2.loc[i,'longitude']=a_19.loc[i,'longitude']
    

df2.drop(df2[np.isnan(df2.longitude) ==True].index, inplace=True)
df2.drop(df2[np.isnan(df2.latitude) ==True].index, inplace=True)
df2.drop(df2[np.isnan(df2.p_nom) ==True].index, inplace=True)
df2.drop(df2[df2.carrier == 'ror' ].index, inplace=True)
df2.drop(df2[df2.carrier == 'Geothermal' ].index, inplace=True)


df2.index=range(len(df2))

year_added=[]
year_removed=[]
for i in range(len(df2)):
    year_added.append( int(df2[df2.columns[0]][i][:4]) )
    year_removed.append( int(df2[df2.columns[0]][i][:4]) +25)

df2.year_added=year_added
df2.year_removed=year_removed   

In [55]:
df2

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,2019,2044,solar,0.086940,49.655942,10.332874
1,2019,2044,solar,0.006000,51.072527,13.043342
2,2019,2044,solar,0.007470,49.198889,12.391779
3,2019,2044,solar,0.009720,49.598032,9.574454
4,2019,2044,solar,0.009990,49.613507,9.480866
...,...,...,...,...,...,...
92317,2019,2044,solar,0.009920,48.867734,10.113042
92318,2019,2044,solar,0.009750,49.821914,10.019952
92319,2019,2044,solar,0.006500,51.410735,9.109868
92320,2019,2044,solar,0.009800,50.275753,9.382804


In [56]:
data=pd.concat([data,df2])
data.index=range(len(data))
data

,year_added,year_removed,carrier,p_nom,latitude,longitude
0,1995,2020,solar,0.001290,49.277654,7.036675
1,1995,2020,solar,0.008800,52.651042,13.708159
2,1995,2020,solar,0.002600,50.742935,11.342881
3,1995,2020,onwind,0.080000,53.127583,12.132579
4,1995,2020,solar,0.003960,48.793771,9.528908
...,...,...,...,...,...,...
1872551,2019,2044,solar,0.009920,48.867734,10.113042
1872552,2019,2044,solar,0.009750,49.821914,10.019952
1872553,2019,2044,solar,0.006500,51.410735,9.109868
1872554,2019,2044,solar,0.009800,50.275753,9.382804


In [59]:
#25 minurtos y no termino.
import Model_Code.utils as ut
from math import pi
"""" Mapping """
data['bus']=data.latitude*0
a=n.buses[['x','y']]
a['bus']=a.index
a.index=range(len(a))
a.y*=pi/180
a.x*=pi/180
data.longitude*=pi/180
data.latitude*=pi/180
""""""

data['bus'] = ut.calculate_nearest_bus(data, a, n)

KeyboardInterrupt: 

In [69]:
import numpy as np
import pandas as pd

def calculate_nearest_bus3(data, a, n):
    """
    Optimized function to calculate the nearest bus for each entry in `data` using vectorized Haversine formula.

    Parameters:
    - data : DataFrame
        Contains 'latitude' and 'longitude' columns (in degrees).
    - a : DataFrame
        Contains 'x', 'y', and 'bus' columns with bus coordinates (in degrees).
    - n : Network
        Network object to filter out only relevant buses for the given carrier.

    Returns:
    - pd.Series with the nearest bus for each location.
    """

    R = 6371  # Earth radius in km
    
    # Convert coordinates to NumPy arrays (convert to radians)
    lat1 = np.radians(data.latitude.values).reshape(-1, 1)  # Shape (N, 1)
    lon1 = np.radians(data.longitude.values).reshape(-1, 1)  # Shape (N, 1)

    lat2 = np.radians(a.y.values).reshape(1, -1)  # Shape (1, M)
    lon2 = np.radians(a.x.values).reshape(1, -1)  # Shape (1, M)

    # Compute Haversine distance matrix (vectorized)
    sin_lat1, sin_lat2 = np.sin(lat1), np.sin(lat2)
    cos_lat1, cos_lat2 = np.cos(lat1), np.cos(lat2)
    cos_lon_diff = np.cos(lon1 - lon2)

    distances = np.arccos(sin_lat1 * sin_lat2 + cos_lat1 * cos_lat2 * cos_lon_diff) * R

    # Convert distances to Pandas DataFrame
    distance_df = pd.DataFrame(distances, index=data.index, columns=a.bus.values)

    # Para cada planta, obtener solo los buses relevantes según su carrier
    nearest_buses = []
    for idx, carrier in data.carrier.iteritems():
        wanted_buses = n.generators.bus[n.generators.carrier == carrier].unique()  # Filtrar buses por carrier
        filtered_distances = distance_df.loc[idx, wanted_buses]  # Considerar solo buses relevantes
        nearest_buses.append(filtered_distances.idxmin())  # Asignar el bus con menor distancia

    return pd.Series(nearest_buses, index=data.index)


In [67]:
data.bus.value_counts()

DE0 1    1872556
Name: bus, dtype: int64

In [70]:
import Model_Code.utils as ut
from math import pi
"""" Mapping """
data['bus']=data.latitude*0
a=n.buses[['x','y']]
a['bus']=a.index
a.index=range(len(a))
a.y*=pi/180
a.x*=pi/180
data.longitude*=pi/180
data.latitude*=pi/180
data['bus'] = calculate_nearest_bus3(data, a, n)

In [72]:
data.bus.value_counts()

DE0 1    1871151
DE0 9       1405
Name: bus, dtype: int64

In [83]:
data[5000:10000]

,year_added,year_removed,carrier,p_nom,latitude,longitude,bus
5000,1998,2023,solar,0.00198,1.486592e-09,3.803950e-10,DE0 1
5001,1998,2023,solar,0.00192,1.398580e-09,2.399838e-10,DE0 1
5002,1998,2023,solar,0.00198,1.406946e-09,2.788361e-10,DE0 1
5003,1998,2023,solar,0.00216,1.449991e-09,1.927199e-10,DE0 1
5004,1998,2023,onwind,0.50000,1.496216e-09,3.118710e-10,DE0 1
...,...,...,...,...,...,...,...
9995,1999,2024,solar,0.00210,1.503157e-09,2.431966e-10,DE0 1
9996,1999,2024,solar,0.00252,1.394563e-09,2.076231e-10,DE0 1
9997,1999,2024,solar,0.00120,1.433340e-09,1.743611e-10,DE0 1
9998,1999,2024,onwind,1.65000,1.526081e-09,2.779290e-10,DE0 1


In [76]:
a

,x,y,bus
0,0.158063,0.849952,DE0 1
1,0.194305,0.855564,DE0 2
2,0.234230,0.917825,DE0 4
3,0.152716,0.877590,DE0 7
4,0.225580,0.941780,DE0 8
5,0.156716,0.919702,DE0 9
6,0.129602,0.897185,DE0 A
7,0.129283,0.871378,DE0 B
8,0.122029,0.862271,DE0 C
9,0.233858,0.892905,DE0 D


## Res biomas

In [9]:
import pandas as pd
data = pd.read_csv('data/renewable_power_plants_DE_bio.csv')
data = data[['commissioning_date', 'decommissioning_date',
                'technology', 'electrical_capacity', 'lat', 'lon', 'postcode']]
data.columns = ['year_added', 'year_removed',
                'carrier', 'p_nom', 'latitude',
                'longitude', 'postcode']
data.carrier = data.carrier.replace('Sewage and landfill gas', 'biomass')
data.carrier = data.carrier.replace('Biomass and biogas', 'biomass')
data.drop(data[data.carrier != 'biomass'].index, inplace=True)
data.index = range(len(data))

In [5]:
data

,year_added,year_removed,carrier,p_nom,latitude,longitude,postcode
0,1900-01-02,NaN,biomass,0.080,48.292429,8.578018,78727.0
1,1905-06-27,NaN,biomass,1.053,48.843981,10.637540,86733.0
2,1911-01-01,NaN,biomass,0.500,48.958223,9.119153,74321.0
3,1966-09-01,NaN,biomass,28.000,49.533744,8.495851,68305.0
4,1969-01-01,NaN,biomass,19.470,49.972251,9.051289,63811.0
...,...,...,...,...,...,...,...
14673,2019-11-14,NaN,biomass,0.099,52.013579,7.182540,48720.0
14674,2019-12-12,NaN,biomass,0.140,48.128224,8.324846,78112.0
14675,2019-12-15,NaN,biomass,0.050,49.011974,12.806920,94379.0
14676,2019-12-17,NaN,biomass,0.075,51.951513,6.847469,46354.0


In [9]:
import numpy as np
year_added = []
year_removed = []
for i in range(len(data)):
    year_added.append(int(data[data.columns[0]][i][:4]))
    year_removed.append(int(data[data.columns[0]][i][:4]) + 30)

data.year_added = year_added
data.year_removed = year_removed

data.drop(data[data.year_added < 1989].index, inplace=True)
data.drop(data[data.year_added > 2020].index, inplace=True)

data.drop(data[np.isnan(data.longitude) == True].index, inplace=True)
data.drop(data[np.isnan(data.latitude) == True].index, inplace=True)
data.drop(data[np.isnan(data.p_nom) == True].index, inplace=True)
data.drop(data[np.isnan(data.postcode) == True].index, inplace=True)

data['bus'] = 0

In [10]:
data

,year_added,year_removed,carrier,p_nom,latitude,longitude,postcode,bus
24,1989,2019,biomass,1.600,53.821017,10.641693,23560.0,0
25,1989,2019,biomass,0.600,53.821017,10.641693,23560.0,0
26,1989,2019,biomass,0.120,48.520552,8.066034,77704.0,0
27,1989,2019,biomass,1.300,54.056257,10.310818,23824.0,0
28,1989,2019,biomass,0.200,51.623072,7.635920,59192.0,0
...,...,...,...,...,...,...,...,...
14673,2019,2049,biomass,0.099,52.013579,7.182540,48720.0,0
14674,2019,2049,biomass,0.140,48.128224,8.324846,78112.0,0
14675,2019,2049,biomass,0.050,49.011974,12.806920,94379.0,0
14676,2019,2049,biomass,0.075,51.951513,6.847469,46354.0,0


In [12]:
import os
nuts_plz = pd.read_csv(os.path.join("data", "plz_nuts3.csv"),sep=';')

In [15]:
nuts_plz

,NUTS3,CODE
0,'DEB15','55774'
1,'DE949','49777'
2,'DEB15','55626'
3,'DEA2B','51465'
4,'DE949','49744'
...,...,...
8327,'DEA1A','42399'
8328,'DEA35','48301'
8329,'DEB12','56656'
8330,'DE212','81248'
